# 0002_signature_from_callable

## Expand cell

In [ ]:
# from IPython.core.display import display, HTML # a depreciated import
from IPython.display import display, HTML 

In [ ]:
display(HTML("<style>.container { width:100% !important; }</style>"))

## Imports and initiate

In [ ]:
from fastdebug.core import *
from fastcore.meta import *

In [ ]:
g = locals()
fdb = Fastdb(inspect._signature_from_callable, outloc=g)
fdbF = Fastdb(FixSigMeta, outloc=g)

## Examples

In [ ]:
from fastdebug.utils import whatinside

In [ ]:
inspect._signature_from_callable(whatinside, sigcls=inspect.Signature)

<Signature (mo, dun: bool = False, func: bool = False, clas: bool = False, bltin: bool = False, lib: bool = False, cal: bool = False)>

In [ ]:
fdb.eg = "inspect._signature_from_callable(whatinside, sigcls=inspect.Signature)"

fdb.eg = """
class Base: # pass
    def __new__(self, **args): pass  # defines a __new__ 

class Foo_new(Base):
    def __init__(self, d, e, f): pass
    
pprint(inspect._signature_from_callable(Foo_new, sigcls=inspect.Signature))
"""
fdb.eg = """
class Base: # pass
    def __new__(self, **args): pass  # defines a __new__ 

class Foo_new_fix(Base, metaclass=FixSigMeta):
    def __init__(self, d, e, f): pass
    
pprint(inspect._signature_from_callable(Foo_new_fix, sigcls=inspect.Signature))
"""

fdb.eg = """
class BaseMeta(type): 
    # using __new__ from type
    def __call__(cls, *args, **kwargs): pass
class Foo_call(metaclass=BaseMeta): 
    def __init__(self, d, e, f): pass

pprint(inspect._signature_from_callable(Foo_call, sigcls=inspect.Signature))
"""

fdbF.eg = """
class BaseMeta(FixSigMeta): 
    # using __new__ of  FixSigMeta instead of type
    def __call__(cls, *args, **kwargs): pass

class Foo_call_fix(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass

pprint(inspect._signature_from_callable(Foo_call_fix, sigcls=inspect.Signature))    
"""

fdb.eg = """
class Foo_init:
    def __init__(self, a, b, c): pass

pprint(inspect._signature_from_callable(Foo_init, sigcls=inspect.Signature))
"""

In [ ]:
fdbF.docsrc(2, "how does a metaclass create a class instance; what does super().__new__() do here;", "inspect.getdoc(super)")
fdbF.docsrc(4, "how to remove self from a signature; how to check whether a class' __init__ is inherited from object or not;",\
            "res", "res.__init__ is not object.__init__")
fdbF.docsrc(1, "Any class having FixSigMeta as metaclass will have its own __init__ func stored in its attr __signature__;\
FixSigMeta uses its __new__ to create a class instance; then check whether its class instance has its own __init__;\
if so, remove self from the sig of __init__; then assign this new sig to __signature__ for the class instance;")

========================================================     Investigating FixSigMeta     ========================================================
===============================================================     on line 2     ================================================================
     with example 
class BaseMeta(FixSigMeta): 
    # using __new__ of  FixSigMeta instead of type
    def __call__(cls, *args, **kwargs): pass

class Foo_call_fix(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass

pprint(inspect._signature_from_callable(Foo_call_fix, sigcls=inspect.Signature))    
     

print selected srcline with expands below--------
class FixSigMeta(type):                                                                                                                                 (0)
    "A metaclass that fixes the signature on classes that override `__new__`"                                                                           (1)
    def __new__(cls, 

In [ ]:
fdbF.snoop()

23:01:29.15 >>> Call to FixSigMeta.__new__ in File "/tmp/FixSigMeta.py", line 5
23:01:29.15 .......... cls = <class '__main__.BaseMeta'>
23:01:29.15 .......... name = 'Foo_call_fix'
23:01:29.15 .......... bases = ()
23:01:29.15 .......... dict = {'__module__': '__main__', '__qualname__': 'Foo_call_fix', '__init__': <function Foo_call_fix.__init__>}
23:01:29.15 .......... len(dict) = 3
23:01:29.15 .......... __class__ = <class 'fastcore.meta.FixSigMeta'>
23:01:29.15    5 |     def __new__(cls, name, bases, dict):
23:01:29.15    6 |         res = super().__new__(cls, name, bases, dict)
23:01:29.15 .............. res = <class '__main__.Foo_call_fix'>
23:01:29.15    7 |         if res.__init__ is not object.__init__: res.__signature__ = _rm_self(inspect.signature(res.__init__))
23:01:29.15    8 |         return res
23:01:29.15 <<< Return value from FixSigMeta.__new__: <class '__main__.Foo_call_fix'>


========================================================     Investigating FixSigMeta     ========================================================
==============================================================     on line None     ==============================================================
     with example 
class BaseMeta(FixSigMeta): 
    # using __new__ of  FixSigMeta instead of type
    def __call__(cls, *args, **kwargs): pass

class Foo_call_fix(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass

pprint(inspect._signature_from_callable(Foo_call_fix, sigcls=inspect.Signature))    
     

<Signature (d, e, f)>


In [ ]:
fdb.docsrc(29, "How to check whether a class has __signature__?", "hasattr(obj, '__signature__')")
fdb.docsrc(82, "how to check whether obj whose signature is builtins;", "inspect.getdoc(_signature_is_builtin)")
fdb.docsrc(7, "inspect.signature is calling inspect._signature_from_callable; \
create _get_signature_of using functools.partial to call on _signature_from_callable itself;\
obj is first tested for callable; then test obj for classmethod; then unwrap to the end unless obj has __signature__;\
if obj has __signature__, assign __signature__ to sig; then test obj for function, is true calling _signature_from_function; \
then test obj whose signature is builtins or not; test whether obj created by functools.partial; test obj is a class or not; \
if obj is a class, then check obj has its own __call__ first; then its own __new__; then its own __init__; then inherited __new__; \
finally inherited __init__; and then get sig from either of them by calling _get_signature_of on them; \
FixSigMeta assigns __init__ function to __signature__ attr for the instance class it creates; \
so that class with FixSigMeta as metaclass can have sig from __init__ through __signature__; \
no more worry about interference of sig from __call__ or __new__.")

=================================================     Investigating _signature_from_callable     =================================================
===============================================================     on line 29     ===============================================================
     with example 
class Foo_init:
    def __init__(self, a, b, c): pass

pprint(inspect._signature_from_callable(Foo_init, sigcls=inspect.Signature))
     

print selected srcline with expands below--------
    # Was this function wrapped by a decorator?                                                                                                         (27)
    if follow_wrapper_chains:                                                                                                                           (28)
        obj = unwrap(obj, stop=(lambda f: hasattr(f, "__signature__")))=================================================================================(29)
                           

In [ ]:
fdb.snoop()

23:01:29.21 >>> Call to _signature_from_callable in File "/tmp/_signature_from_callable.py", line 3
23:01:29.21 ...... obj = <class '__main__.Foo_init'>
23:01:29.21 ...... follow_wrapper_chains = True
23:01:29.21 ...... skip_bound_arg = True
23:01:29.21 ...... sigcls = <class 'inspect.Signature'>
23:01:29.21    3 | def _signature_from_callable(obj, *,
23:01:29.21   12 |     _get_signature_of = functools.partial(_signature_from_callable,
23:01:29.21   13 |                                 follow_wrapper_chains=follow_wrapper_chains,
23:01:29.21   14 |                                 skip_bound_arg=skip_bound_arg,
23:01:29.21   15 |                                 sigcls=sigcls)
23:01:29.21   12 |     _get_signature_of = functools.partial(_signature_from_callable,
23:01:29.21 .......... _get_signature_of = functools.partial(<function _signature_from_call...und_arg=True, sigcls=<class 'inspect.Signature'>)
23:01:29.21   17 |     if not callable(obj):
23:01:29.22   20 |     if isinstance(ob

=================================================     Investigating _signature_from_callable     =================================================
==============================================================     on line None     ==============================================================
     with example 
class Foo_init:
    def __init__(self, a, b, c): pass

pprint(inspect._signature_from_callable(Foo_init, sigcls=inspect.Signature))
     



23:01:29.35 !!! AttributeError: type object 'Foo_init' has no attribute '__signature__'
23:01:29.35 !!! When getting attribute: obj.__signature__
23:01:29.35   41 |     except AttributeError:
23:01:29.35   42 |         pass
23:01:29.35   51 |     try:
23:01:29.35   52 |         partialmethod = obj._partialmethod
23:01:29.36 !!! AttributeError: type object 'Foo_init' has no attribute '_partialmethod'
23:01:29.36 !!! When getting attribute: obj._partialmethod
23:01:29.36   53 |     except AttributeError:
23:01:29.36   54 |         pass
23:01:29.36   79 |     if isfunction(obj) or _signature_is_functionlike(obj):
23:01:29.36   85 |     if _signature_is_builtin(obj):
23:01:29.36   89 |     if isinstance(obj, functools.partial):
23:01:29.36   93 |     sig = None
23:01:29.36   94 |     if isinstance(obj, type):
23:01:29.36   99 |         call = _signature_get_user_defined_method(type(obj), '__call__')
23:01:29.36 .............. call = None
23:01:29.36  100 |         if call is not None:
23:0

<Signature (a, b, c)>


In [ ]:
fdbF.print()

========================================================     Investigating FixSigMeta     ========================================================
==============================================================     on line None     ==============================================================
     with example 
class BaseMeta(FixSigMeta): 
    # using __new__ of  FixSigMeta instead of type
    def __call__(cls, *args, **kwargs): pass

class Foo_call_fix(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass

pprint(inspect._signature_from_callable(Foo_call_fix, sigcls=inspect.Signature))    
     

class FixSigMeta(type):===================================================================(0)       
    "A metaclass that fixes the signature on classes that override `__new__`"=============(1) # Any class having FixSigMeta as metaclass will have its own __init__ func stored in its attr __signature__;FixSigMeta uses its __new__ to create a class instance; then check whether its c

In [ ]:
fdb.print(30, 1)

def _signature_from_callable(obj, *,======================================================(0)       
                             follow_wrapper_chains=True,==================================(1)       
                             skip_bound_arg=True,=========================================(2)       
                             sigcls):=====================================================(3)       
                                                                                                                                                        (4)
    """Private helper function to get signature for arbitrary=============================(5)       
    callable objects.=====================================================================(6)       
    """===================================================================================(7) # inspect.signature is calling inspect._signature_from_callable; create _get_signature_of using functools.partial to call on _signature_from_cal

In [ ]:
#| hide
!jupytext --to md /Users/Natsume/Documents/fastdebug/Demos/_signature_from_callable_with_FixSigMeta.ipynb
!mv /Users/Natsume/Documents/fastdebug/Demos/_signature_from_callable_with_FixSigMeta.md \
/Users/Natsume/Documents/divefastai/Debuggable/jupytext/

!jupyter nbconvert --config /Users/Natsume/Documents/mynbcfg.py --to markdown \
--output-dir /Users/Natsume/Documents/divefastai/Debuggable/nbconvert

[jupytext] Reading /Users/Natsume/Documents/fastdebug/Demos/_signature_from_callable_with_FixSigMeta.ipynb in format ipynb
Traceback (most recent call last):
  File "/Users/Natsume/mambaforge/bin/jupytext", line 10, in <module>
    sys.exit(jupytext())
  File "/Users/Natsume/mambaforge/lib/python3.9/site-packages/jupytext/cli.py", line 488, in jupytext
    exit_code += jupytext_single_file(nb_file, args, log)
  File "/Users/Natsume/mambaforge/lib/python3.9/site-packages/jupytext/cli.py", line 552, in jupytext_single_file
    notebook = read(nb_file, fmt=fmt, config=config)
  File "/Users/Natsume/mambaforge/lib/python3.9/site-packages/jupytext/jupytext.py", line 411, in read
    with open(fp, encoding="utf-8") as stream:
FileNotFoundError: [Errno 2] No such file or directory: '/Users/Natsume/Documents/fastdebug/Demos/_signature_from_callable_with_FixSigMeta.ipynb'
mv: rename /Users/Natsume/Documents/fastdebug/Demos/_signature_from_callable_with_FixSigMeta.md to /Users/Natsume/Documents/